# COMP9033 - Data Analytics Lab 08b: Decision tree regression
## Introduction

This lab focuses on anomaly detection using decision tree and random forest regression. It's a direct counterpart to the linear regression modelling in Lab 06 and the $k$ nearest neighbours regression in Lab 07b. At the end of the lab, you should be able to use `scikit-learn` to:

- Create a decision tree regression model and a random forest regression model.
- Use the models to predict new values.
- Measure the accuracy of the models.

### Getting started

Let's start by importing the packages we'll need. As usual, we'll import `pandas` for exploratory analysis, but this week we're also going to use the `tree` subpackage from `scikit-learn` to create decision tree models and the `ensemble` subpackage to create random forest models.

In [ ]:
%matplotlib inline
import pandas as pd

from math import sqrt # We'll need this later

from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeRegressor

Next, let's load the data. Write the path to your `server_load_historical.csv` file in the cell below:

In [ ]:
path = "data/server_load_historical.csv"

The data we're loading is simulated network interface load data (in Gb/s) from a front end web server:

In [ ]:
df = pd.read_csv(path, parse_dates=True, index_col="time")
df.head()

The data is noisy due to fluctuating user logins and data transfers and follows a seasonal trend:

In [ ]:
print(df.describe()) # Print summary statistics
df.plot(); # Plot the data

## Data modelling

Let's build a decision tree regression model of the server load data. `scikit-learn` supports decision tree functionality via the [`tree`](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.tree) subpackage. This subpackage supports both decision tree regression and classification. We can use the [`DecisionTreeRegressor`](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html#sklearn.tree.DecisionTreeRegressor) class to build our model.

As before, we want to use our existing knowledge of server load to predict how it will behave in the future. Specifically, we're interested in modelling the behaviour of the server load over time. Therefore, our predictor variable will be the time of day and our target variable will be the server load.

Again, to get the time of day, we can subtract the first timestamp in our index from each timestamp in our index and convert the results to integer values:

In [ ]:
time = (df.index - df.index[0]).astype('int') # Computes the time from midnight in nanoseconds

Next, we construct our feature matrix, $\mathbf{X}$, using our predictor values:

In [ ]:
X = pd.DataFrame({'time': time}, index=df.index)
X.head()

And we set our target variable, $\mathbf{y}$, to be the server load data:

In [ ]:
y = df['server_load']
y.head()

Let's use hold out validation to estimate our model error. We can use the [`train_test_split`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) method (just like last week's lab) to split the data into a training set and a test set. Let's use 20% of the data for testing:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Decision tree regression

`DecisionTreeRegressor` accepts a number of different hyperparameters and the model we build may be more or less accurate depending on their values. We can get a list of these modelling parameters using the `get_params` method of the estimator (this works on any `scikit-learn` estimator), like this:

In [ ]:
DecisionTreeRegressor().get_params()

You can find a more detailed description of each parameter in the `scikit-learn` [documentation](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html#sklearn.tree.DecisionTreeRegressor). 

Let's build our decision tree model using a single stage pipeline and a grid search. Note how, like $k$ nearest neighbours, we don't need to do any scaling or polynomial feature generation like we did with the linear regression model in Lab 06.

In [ ]:
pipeline = make_pipeline(
    DecisionTreeRegressor(random_state=0) # Control randomness, so the lab works the same for everyone
)

parameters = {
    'decisiontreeregressor__criterion': ['mse', 'mae'],
    'decisiontreeregressor__min_samples_leaf': [1, 5],
    'decisiontreeregressor__min_samples_split': [2, 5] # 2 is the minimum
}

gs = GridSearchCV(pipeline, parameters, cv=5) # Use 5 fold cross validation when selecting the best model
gs.fit(X_train, y_train); # Fit using the training set

We can check how well our model works by using it to predict values for the test set and measuring the error:

In [ ]:
# Make predictions about the test data
y_pred = gs.predict(X_test)

# Print error measurementsa
print('MAE: %.2f Gb/s' % mean_absolute_error(y_test, y_pred))
print('RMSE: %.2f Gb/s' % sqrt(mean_squared_error(y_test, y_pred))) # Use sqrt to get the RMSE from the MSE

# Plot the predictions
ax = df.plot()

predicted = pd.DataFrame({'predicted': y_pred}, index=X_test.index)
predicted.plot(ax=ax, marker='o', linewidth=0);

The decision tree model fits the overall trend of the server load data, but there is quite a lot of variance from sample to sample. While we could build a better model using a more exhaustive grid search (try this for yourself), let's use random forest regression instead to see if we can make an improvement.

### Random forest regression

Decision tree regression works well in lots of cases, but tends to be unstable near decision boundaries. Let's use random forests to build an alternative regression model and see what the difference is.

`scikit-learn` supports ensemble model functionality via the [`ensemble`](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.ensemble) subpackage. This subpackage supports random forest regression and classification, as well as several other ensemble modelling algorithms. We can use the [`RandomForestRegressor`](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor) class to build our model.

When building random forest regressors, we can choose to specify the same hyperparameters as with decision tree regressors. Let's also specify a number of different forest sizes in our grid search. We can do this using the `n_estimators` attribute:

In [ ]:
pipeline = make_pipeline(
    RandomForestRegressor(random_state=0)
)

parameters = {
    'randomforestregressor__n_estimators': [2, 5, 10, 15, 20],
    'randomforestregressor__criterion': ['mse', 'mae'],
    'randomforestregressor__min_samples_leaf': [1, 5],
    'randomforestregressor__min_samples_split': [2, 5] # 2 is the minimum
}

gs = GridSearchCV(pipeline, parameters, cv=5)
gs.fit(X_train, y_train); # Fit using the training set

# Make predictions about the test data
y_pred = gs.predict(X_test)

# Print error measurementsa
print('MAE: %.2f Gb/s' % mean_absolute_error(y_test, y_pred))
print('RMSE: %.2f Gb/s' % sqrt(mean_squared_error(y_test, y_pred))) # Use sqrt to get the RMSE from the MSE

# Plot the predictions
ax = df.plot()

predicted = pd.DataFrame({'predicted': y_pred}, index=X_test.index)
predicted.plot(ax=ax, marker='o', linewidth=0);

As can be seen, the random forest model better fits the trend of the server load data and has significantly lower error than the decision tree regressor model. Let's use the random forest model to construct an anomaly detector to automatically flag unusual server load activity in the future.

## Anomaly detection

Now that we have an accurate model, we can use it to detect anomalies in new data. Let's load some data where the server experiences an unusual load for the time of day:

In [ ]:
path = 'data/server_load_new.csv'

In [ ]:
df2 = pd.read_csv(path, parse_dates=True, index_col='time')
df2.plot();

We can extract our predictor matrix and our target variable just like before:

In [ ]:
X_new = pd.DataFrame({'time': (df2.index - df2.index[0]).astype('int')}, index=df2.index)
y_new = df2['server_load']

We can see the difference between the new data and the model by plotting like before:

In [ ]:
# Make predictions about the test data
y_pred = gs.predict(X_new)

# Plot the predictions
ax = df2.plot()

predicted = pd.DataFrame({'predicted': y_pred}, index=X_new.index)
predicted.plot(ax=ax);

While it is easy to gauge the anomaly visually, ideally an automatic system would detect it for us. This way, we don't have to micro-manage systems; instead, we can just sit back and wait to be alerted. Let's use a standard score test to find observations that are extremely different from what we expect.

The standard score test works by finding observations ($x_i$) in a sample ($X = \{x_1, x_2, \ldots, x_n\}$) that are more than a given number ($\lambda$) of standard deviations ($\sigma_x$) away from the average value ($\bar{x}$), i.e.

\begin{align}
    \left| z(x_i) \right| > \lambda,
\end{align}
where
\begin{align}
    z(x_i) = \frac{x_i - \bar{x}}{\sigma_x}.
\end{align}

We can combine the equations above to work out the range of non-outlying observations, i.e.

\begin{align}
    x_i \in [\bar{x} - \lambda \sigma_x, \bar{x} + \lambda \sigma_x].
\end{align}

Consequently, any observation that lies outside this range can be considered an anomaly.

In our case, we want to detect observations that deviate significantly from the predicted value given by our model. We can do this by checking for model prediction errors that lie outside the range of non-outlying prediction errors. Let's use $\lambda = 3$ to find instances where the model prediction error is more than three standard deviations from the mean prediction error:

In [ ]:
# The prediction error on the training set represents the "normal" error we should expect
error = y_train - gs.predict(X_train)

# Compute the range of non-outlying prediction errors
l = 3 # l = lambda
min_error = error.mean() - l * error.std()
max_error = error.mean() + l * error.std()

Next, we compute the prediction error on the new data:

In [ ]:
error = y_new - gs.predict(X_new)

Plotting the output, we can see exactly where an automated system would have shown us that an anomaly was occurring:

In [ ]:
ax = df2.plot()
ax.fill_between(df2.index, y_pred + min_error, y_pred + max_error, color='green', alpha=0.5)
ax.fill_between(df2.index, ax.get_ylim()[0], ax.get_ylim()[1], color='red', alpha=0.5,
                where=(error < min_error) | (error > max_error));

As can be seen, the system detects the server load anomaly that occurs after 18:00, but it also catches two other anomalies, one just before 06:00 and the other just after 12:00. In these cases, the anomaly detector has misclassified the data as anomalous (i.e. it has made a mistake). We can increase our value of $\lambda$ to ensure that this doesn't happen, at the cost of detecting fewer genuine anomalies. Alternatively, we can accept that the error rate is small enough to be tolerable, and perhaps suppress alerts for transient anomalies with short lifespans.